In [1]:
import sys
from pathlib import Path
import numbers
import random
import json
from itertools import chain
print(sys.executable)
print(sys.version)
sys.path.extend(['..', '../..'])
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.ensemble import IsolationForest
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from skimage import transform, color
import cv2
import imageio
import umap
from quant.experiment.clustering import Clustering
from base.data.wsi_reader import WSIReader

/well/rittscher/users/achatrian/.conda/envs/pyenvclone/bin/python
3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [2]:
e = Clustering('guassians',
              ('MinMaxScaler', 'StandardScaler', 'PCA', 'GaussianMixture'),
              (MinMaxScaler(), StandardScaler(), PCA(n_components=10), GaussianMixture(n_components=4)),
               outlier_removal=IsolationForest(contamination=0.05),
               caching_path='/well/rittscher/users/achatrian/pipeline_cache')

In [3]:
# e.read_data_from_dir('/well/rittscher/projects/prostate-gland-phenotyping/WSI/data/features', max_memory_use=1e10)

In [4]:
# e.x.to_hdf('/well/rittscher/projects/prostate-gland-phenotyping/WSI/data/features/all.h5', 'x')

In [5]:
e.x = pd.read_hdf('/well/rittscher/projects/prostate-gland-phenotyping/WSI/data/features/all.h5', 'x')
e.x.shape

(19347, 7100)

In [6]:
e.x.columns[0:76]

Index(['outer_hu_moment0', 'outer_hu_moment1', 'outer_hu_moment2',
       'outer_hu_moment3', 'outer_hu_moment4', 'outer_hu_moment5',
       'outer_hu_moment6', 'outer_eccentricity', 'outer_solidity',
       'outer_extent', 'outer_inertia_eigenval0', 'outer_inertia_eigenval1',
       'inner_hu_moment0', 'inner_hu_moment1', 'inner_hu_moment2',
       'inner_hu_moment3', 'inner_hu_moment4', 'inner_hu_moment5',
       'inner_hu_moment6', 'inner_eccentricity', 'inner_solidity',
       'inner_extent', 'inner_inertia_eigenval0', 'inner_inertia_eigenval1',
       'gray_haralick_vert0', 'gray_haralick_vert1', 'gray_haralick_vert2',
       'gray_haralick_vert3', 'gray_haralick_vert4', 'gray_haralick_vert5',
       'gray_haralick_vert6', 'gray_haralick_vert7', 'gray_haralick_vert8',
       'gray_haralick_vert9', 'gray_haralick_vert10', 'gray_haralick_vert11',
       'gray_haralick_vert12', 'gray_haralick_horz0', 'gray_haralick_horz1',
       'gray_haralick_horz2', 'gray_haralick_horz3', 'gray_ha

In [7]:
to_reg_prop = 24
to_haralick=76
e.x = e.x.iloc[:, 0:to_reg_prop]
e.x.shape

(19347, 24)

In [8]:
remove_outliers = True
if remove_outliers:
    e.remove_outliers()

/well/rittscher/users/achatrian/.conda/envs/pyenvclone/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/well/rittscher/users/achatrian/.conda/envs/pyenvclone/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


Removed 968 outliers through IsolationForest(behaviour='old', bootstrap=False, contamination=0.05,
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=None, verbose=0, warm_start=False)


In [9]:
print("Num points ", len(e.x))
print("Num fields ", len(e.x.columns))
if hasattr(e.x.index, 'levels'):
    print(e.x.index.levels[0])
print(e.x.columns)

Num points  18379
Num fields  24
Index(['17_A047-10719_16L+-+2017-05-11+08.56.52',
       '17_A047-19575_162E+-+2017-05-11+08.34.36',
       '17_A047-28956_156N+-+2017-05-11+09.57.53',
       '17_A047-4463_153D+-+2017-05-11+09.40.22',
       '17_A047-4519_1614P+-+2017-05-11+09.50.49',
       '17_A047-47800_15J+-+2017-05-11+08.12.44',
       '17_A047-8544_16J+-+2017-05-11+08.49.27', 'EU_18387_14_1E_HandE',
       'EU_2248_16_K_H&E', 'EU_26162_16_9x_HandE+-+2017-11-28+11.03.04',
       'EU_29542_16_1S_HandE+-+2017-11-28+12.49.46',
       'EU_29797_14_5F_HandE+-+2017-11-28+13.15.48',
       'EU_30479_16_G_HandE+-+2017-11-28+12.31.57',
       'EU_30831_14_4D_HandE+-+2017-11-28+11.15.23',
       'EU_36244_16_9A_HandE+-+2017-11-28+12.41.47',
       'EU_37106_15_2K_HandE+-+2017-11-28+11.43.55',
       'EU_38663_17_7G_HandE+-+2017-11-28+11.48.55',
       'EU_42628_15_4J_HandE+-+2017-11-28+12.20.38',
       'EU_49057_14_2F_HandE+-+2017-11-28+10.42.55',
       'EU_52005_15_2I_HandE+-+2017-11-28+

In [10]:
e.run(store_original=True)

Total run time: 1.111720085144043s


TypeError: '>' not supported between instances of 'tuple' and 'int'

In [ ]:
e.plot_clusters() #colors=list(str(i * 0.1) for i in range(10)))

In [ ]:
colors = list(str(i * 0.05) for i in range(20))
for i, slide_id in enumerate(path.with_suffix('').name for path in e.loaded_paths):
    plt.scatter(e.x.loc[slide_id, 'PCA0'], e.x.loc[slide_id, 'PCA1'], color=colors[i])

In [ ]:
data_dir = Path('/well/rittscher/projects/prostate-gland-phenotyping/WSI/data/features')
e.loaded_paths = tuple(path for path in data_dir.iterdir() if path.suffix == '.json')
print(e.loaded_paths)

In [ ]:
means = e.pipeline.named_steps['GaussianMixture'].means_

In [ ]:
examples = e.get_examples(n_examples=10, cluster_centers=means)

In [ ]:
e.save_examples_grid('/well/rittscher/users/achatrian/temp', examples, image_size=256)

# Label comparison

In [ ]:
# read in csv file
label_file = '/well/rittscher/projects/prostate-gland-phenotyping/WSI/data/other/TA206_Scores_V2_190718.csv'
labels = pd.read_csv(label_file, sep='\t')
labels = labels.set_index('file_name')

In [ ]:
labels

In [ ]:
erg = labels['ERG H Score'].apply(lambda x: int(x) if x.isdigit() else 0)
erg = erg.apply(lambda x: int(x) > 0)
erg = erg.iloc[0::2]  # remove benign samples from spreadseet
erg.index = [s[:-5] for s in erg.index]

In [ ]:
# plot feature space against ERG:
colors = ['r', 'b']
for slide, erg_v in erg.iteritems():  # erg is a Series
    color_ = colors[int(erg_v)]
    try:
        slide_x = e.x.loc[slide]
        plt.scatter(slide_x.iloc[:, 0], slide_x.iloc[:, 1], color=color_)
    except KeyError:
        pass

In [ ]:
# PTEN
pten = labels['PTEN H Score RESCORE BETTER STAINED TMA'].apply(lambda x: int(x) if x.isdigit() else 0)
pten = pten.apply(lambda x: int(x) > 0)
pten = pten.iloc[0::2]  # remove benign samples from spreadseet
pten.index = [s[:-5] for s in pten.index]

In [ ]:
# plot feature space against ERG:
colors = ['r', 'b']
for slide, pten_v in pten.iteritems():  # erg is a Series
    color_ = colors[int(pten_v)]
    try:
        slide_x = e.x.loc[slide]
        plt.scatter(slide_x.iloc[:, 0], slide_x.iloc[:, 1], color=color_)
    except KeyError:
        pass

In [ ]:
# pi
proliferation = labels['Proliferation index Mib-1 0-100% (only invasive tumour scored, not PIN or intraductal Ca or inflammatory cells. In the benign, only luminal ep cells scored)'].apply(lambda x: int(x) if x.isdigit() else 0)
proliferation = proliferation.iloc[0::2]  # remove benign samples from spreadseet
proliferation.index = [s[:-5] for s in proliferation.index]

In [ ]:
np.unique(proliferation)

In [ ]:
# plot feature space against ERG:
colors = {0: 'r', 1: 'b', 2: 'g', 5: 'y', 8: 'k'}
normalized_x = (e.x-e.x.min())/(e.x.max()-e.x.min())
for slide, pr_v in proliferation.iteritems():  # erg is a Series
    color_ = colors[int(pr_v)]
    try:
        slide_x = normalized_x.loc[slide]
        plt.scatter(slide_x.iloc[:, 0], slide_x.iloc[:, 1], color=color_)
    except KeyError:
        pass

In [ ]:
# t-sne
x = MinMaxScaler().fit_transform(e.x)
x = StandardScaler().fit_transform(x)
x_tsne = TSNE(n_components=2).fit_transform(x)
plt.scatter(x_tsne[:, 0], x_tsne[:, 1])

In [ ]:
# t-sne
x = MinMaxScaler().fit_transform(e.x)
x = StandardScaler().fit_transform(x)
x_umap = umap.UMAP(n_neighbors=100).fit_transform(x)
plt.scatter(x_umap[:, 0], x_umap[:, 1])

In [ ]:
x_tsne = pd.DataFrame(x_tsne, index=e.x.index, columns=['tsne0', 'tsne1'])
# plot feature space against ERG:
colors = {0: 'r', 1: 'b', 2: 'g', 5: 'y', 8: 'k'}
for slide, pr_v in proliferation.iteritems():  # erg is a Series
    color_ = colors[int(pr_v)]
    try:
        slide_x = x_tsne.loc[slide]
        plt.scatter(slide_x.iloc[:, 0], slide_x.iloc[:, 1], color=color_)
    except KeyError:
        pass

In [ ]:
# Gleason
gleason_file = '/well/rittscher/projects/prostate-gland-phenotyping/WSI/data/other/gleason.csv'
gleason = pd.read_csv(gleason_file, sep=',')
gleason_map = {'3+4': 0, '4+3': 1, '4+5': 2}
gleason['Gleason'] = gleason['Gleason'].map(lambda s: gleason_map[s])
gleason = gleason.set_index('file_id')

In [ ]:
gleason

In [ ]:
x_tsne = pd.DataFrame(x_tsne, index=e.x.index, columns=['tsne0', 'tsne1'])
# plot feature space against ERG:
colors = {0: 'r', 1: 'b', 2: 'g', 5: 'y', 8: 'k'}
markers = {0: '.', 1: ',', 2: 'x'}
for slide, score in gleason['Gleason'].iteritems():  # erg is a Series
    color_ = colors[int(score)]
    marker_ = markers[int(score)]
    try:
        slide_x = x_tsne.loc[slide[:-5]]
        plt.scatter(slide_x.iloc[:, 0], slide_x.iloc[:, 1], color=color_, marker='.')
    except KeyError:
        pass

In [ ]:
e1 = Clustering('tsne', ['gm'], [GaussianMixture(n_components=4)])
e1.x = x_tsne
e1.run()